In [2]:
import numpy as np
from math import log, sqrt
import nltk
from nltk.corpus import inaugural
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nltk.download('inaugural')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()               #initializing lemmatizer

[nltk_data] Downloading package inaugural to
[nltk_data]     C:\Users\favas\AppData\Roaming\nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\favas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Model-1: IR systems based on TF-IDF Vectorization

###### Rank by TF-IDF score

In [6]:
# define a function for pre-processing the test
# this function can be used with TfidfVectorizer
def preprocess(x):
    words = word_tokenize(x)                                 #text to tokens
    words = [word for word in words if word.isalnum()]       #remove non alphanumeric
    words = [lemmatizer.lemmatize(word) for word in words]   #lemmetize
    words = [lemmatizer.lemmatize(word, pos='a') for word in words] #lemetize adjectives
    return ' '.join(words)                                          #join back to single string and return

In [7]:
# function calculate total tfidf for a document based on query keywords
# this function return a dictionary with keys as fileid and values as tfidfs
# arguments are all the dictionary of text as values and file id as keys and query keywords as a list

def get_tfidf(corpus, keywords):
    
    file_ids = list(corpus.keys())
    texts = corpus.values()
    
    # convert all documents to vectors based on tfidfs of each terms
    vectorizer = TfidfVectorizer(lowercase= True, stop_words='english', preprocessor=preprocess)
    X = vectorizer.fit_transform(texts)
    features = vectorizer.get_feature_names()
    vectors = X.toarray()
    
    #retrieve tfidf correspoding to a keyword in the query from the vectors of each document
    #sum all tfidf corresponding to keywords in the query for all documents
    #save the values in a dictioary and return it
    tfidfs = {}
    for i in range(len(file_ids)):
        tfidf_total = 0
        for j in range(len(keywords)):
            # apply pre-processing to the query keywords
            keyword = keywords[j].lower()
            keyword = lemmatizer.lemmatize(keyword)
            keyword = lemmatizer.lemmatize(keyword, pos='a')
            if keyword in features:
                ind = features.index(keyword)
                tfidf_total = tfidf_total+vectors[i,ind]
        tfidfs[file_ids[i]] = tfidf_total
    return tfidfs
     

In [8]:
# function for retrieving relevant documents
# arguments are a dictionary of texts as values and keys as fileids and numer of keywords in query

def get_relevant_docs_by_tfidf(corpus,num_query_keywords):
    
    # input all the keywords
    print(f'\nPlease enter {num_query_keywords} keywords:')
    keywords = []
    for i in range(num_query_keywords):
        word = input()
        keywords.append(word)
        
    # get tfidfs of each documents corresponding to query keywords
    #dictionary with keys as file ids and values as tfidf scores
    tfidfs = get_tfidf(corpus,keywords)
    
    # sort the the tfidfs in descending order 
    sorted_tfidfs = sorted(tfidfs.items(), key=lambda kv: kv[1], reverse=True)
    best_5_docs = sorted_tfidfs[:5] 
    
    # print best 5 ranked documents
    print('\nRelevant documents:')
    for doc in best_5_docs:
        print(doc[0]) 

In [4]:
# get file ids from the inaugural corpus
# save all documents in a dictionary with key as file ids and corresponding texts as values
file_ids = inaugural.fileids()
corpus = {}
for i in range(len(file_ids)):
    text = inaugural.raw(fileids=file_ids[i])
    corpus[file_ids[i]] = text

In [55]:
# retrieve by 2 keywords
get_relevant_docs_by_tfidf(corpus, num_query_keywords=2)
get_relevant_docs_by_tfidf(corpus, num_query_keywords=2)
get_relevant_docs_by_tfidf(corpus, num_query_keywords=2)
get_relevant_docs_by_tfidf(corpus, num_query_keywords=2)


Please enter 2 keywords:
freedom
jobs

Relevant documents:
2005-Bush.txt
2013-Obama.txt
1957-Eisenhower.txt
2017-Trump.txt
1985-Reagan.txt

Please enter 2 keywords:
slavery
war

Relevant documents:
1865-Lincoln.txt
1813-Madison.txt
1857-Buchanan.txt
1821-Monroe.txt
1881-Garfield.txt

Please enter 2 keywords:
liberty
slavery

Relevant documents:
2005-Bush.txt
1857-Buchanan.txt
1881-Garfield.txt
1965-Johnson.txt
1841-Harrison.txt

Please enter 2 keywords:
freedom
military

Relevant documents:
2005-Bush.txt
1957-Eisenhower.txt
1985-Reagan.txt
1953-Eisenhower.txt
1949-Truman.txt


In [57]:
get_relevant_docs_by_tfidf(corpus,num_query_keywords=4)     # example with 4 keywords


Please enter 4 keywords:
war
weapons
missile
military

Relevant documents:
1813-Madison.txt
1865-Lincoln.txt
1985-Reagan.txt
1921-Harding.txt
1825-Adams.txt


###### Rank by cosine similiarity

In [105]:
# function for tfidf vectorization
def get_tfidf_vectors(texts):
    vectorizer = TfidfVectorizer(lowercase= True, stop_words='english', preprocessor=preprocess)
    vectorizer.fit(texts)
    X = vectorizer.transform(texts)
    features = vectorizer.get_feature_names()
    return X

In [111]:
# function for retrieving relevant documents based on cosine similiarity
# arguments are a dictionary of texts as values and keys as fileids and numer of keywords in query

def get_docs_by_cosine_similiarity(corpus,num_query_keywords):
    
    # input all the keywords in query
    print(f'\nPlease enter {num_query_keywords} keywords:')
    keywords = []
    for i in range(num_query_keywords):
        word = input()
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        word = lemmatizer.lemmatize(word, pos='a')
        keywords.append(word)
      

    query = ' '.join(keywords)                      # join query keywords
    file_ids = list(corpus.keys())                  # get file ids
    texts = list(corpus.values())                   # get documents
    texts.append(query)                             # add query to documents for vectorization
    
        
    
    X = get_tfidf_vectors(texts)                   # get tfidf vectors
    query_vector = X[-1, :]                        # retrieve query vector
    document_vector = X[:-1,:]                     # retrieve document vectors

    
    # calculate cosine similiarity between documents and query
    # save values in a dictionary with keys as file ids
    fileid_similiarity = {}
    for i in range(len(file_ids)):
        similiarity = cosine_similarity(document_vector[i,:],query_vector)
        fileid_similiarity[file_ids[i]] = similiarity[0][0]
    
    
    # sort cosine similiarity in descending order  and retrive best five docs
    sorted_similiarity = sorted(fileid_similiarity.items(), key=lambda kv: kv[1], reverse=True)
    best_5_docs = sorted_similiarity[:5] 
    
    # print best 5 ranked documents
    print('\nRelevant documents:')
    for doc in best_5_docs:
        print(doc[0]) 

In [112]:
get_docs_by_cosine_similiarity(corpus,num_query_keywords=2)
get_docs_by_cosine_similiarity(corpus,num_query_keywords=2)
get_docs_by_cosine_similiarity(corpus,num_query_keywords=2)
get_docs_by_cosine_similiarity(corpus,num_query_keywords=2)


Please enter 2 keywords:
freedom
jobs

Relevant documents:
2005-Bush.txt
2017-Trump.txt
2013-Obama.txt
2009-Obama.txt
1981-Reagan.txt

Please enter 2 keywords:
slavery
war

Relevant documents:
1865-Lincoln.txt
1857-Buchanan.txt
1813-Madison.txt
1881-Garfield.txt
1861-Lincoln.txt

Please enter 2 keywords:
liberty
slavery

Relevant documents:
1857-Buchanan.txt
2005-Bush.txt
1881-Garfield.txt
1861-Lincoln.txt
1889-Harrison.txt

Please enter 2 keywords:
freedom
military

Relevant documents:
2005-Bush.txt
1957-Eisenhower.txt
1985-Reagan.txt
1953-Eisenhower.txt
1949-Truman.txt


#### Model -2: Binary Independance Model

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

#count vectorizer with binary true, only detect presence of word
vectorizer1 = CountVectorizer(stop_words='english',lowercase= True,binary=True,preprocessor=preprocess) 
#count vectorizer with binary true, count tf in the document
vectorizer2 = CountVectorizer(stop_words='english',lowercase= True,binary=False,preprocessor=preprocess)

###### RSV weight of a term t
\begin{equation}
\ W_t = log(\dfrac{p_t}{1-p_t}) + log(\dfrac{1-u_t}{u_t})
\ where, p_t = \dfrac{DF_t}{N+0.5},
\ \dfrac{1-u_t}{u_t} \approx \dfrac{N}{DF_t}
\end{equation}

In [14]:
# calculating rsv weight of each term
# arguments are binary vectors of each documents, feature vector and Number of documents as N

def calculate_rsv_weight(X, features, N):
    term_rsv = {}
    for i in range(len(features)):
        DF = sum(X[:,i])                                 #DF of a term sum of column corresponding to the term
        p = DF/(N+0.5)
        rsv = log(N/DF) + log(p/(1-p))
        term_rsv[features[i]] = rsv
    
    return term_rsv

###### Relevance score of a document for a query

\begin{equation}
\ rel(D/Q) = \sum_{t \in Q} W_t
\end{equation}

In [30]:
def get_docs_by_bim(corpus,num_query_keywords):
    
    # input all the keywords in query\
    # preprocess the keywords
    print(f'\nPlease enter {num_query_keywords} keywords:')
    keywords = []
    for i in range(num_query_keywords):
        word = input()
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        word = lemmatizer.lemmatize(word, pos='a')
        keywords.append(word)
        
        
    texts = corpus.values()
    file_ids = list(corpus.keys())
    X = vectorizer1.fit_transform(texts)                          # binary vectorize the documents
    features = vectorizer1.get_feature_names()
    X = X.toarray()
    N = len(corpus)
    
    term_rsv = calculate_rsv_weight(X, features, N)              # calculate rsv of each terms
    id_rsv = {} 
    
    # calculate relevance of each document by summing all Wt corresponding to the terms in the query
    for i in range(len(file_ids)):
        rsv_total = 0
        for word in keywords:
            if word in features:                             # check keyword in the feature vector
                index = features.index(word)
                if X[i,index] > 0:                           # check word is in the document
                    rsv = term_rsv[word]
                    rsv_total = rsv_total+rsv
                    
        id_rsv[file_ids[i]] = rsv_total                     # save rsv score to the dictionary
    
    # sort rsv scores and retrieve 5 relevant documents
    sorted_rsv = sorted(id_rsv.items(), key=lambda kv: kv[1], reverse=True)
    best_5_docs = sorted_rsv[:5] 
    
    # print best 5 ranked documents
    print('\nRelevant documents:')
    for doc in best_5_docs:
        print(doc[0]) 

In [32]:
get_docs_by_bim(corpus,num_query_keywords=2)
get_docs_by_bim(corpus,num_query_keywords=2)
get_docs_by_bim(corpus,num_query_keywords=2)
get_docs_by_bim(corpus,num_query_keywords=2)


Please enter 2 keywords:
freedom
jobs

Relevant documents:
1981-Reagan.txt
1989-Bush.txt
1993-Clinton.txt
1997-Clinton.txt
2009-Obama.txt

Please enter 2 keywords:
slavery
war

Relevant documents:
1837-VanBuren.txt
1857-Buchanan.txt
1861-Lincoln.txt
1865-Lincoln.txt
1881-Garfield.txt

Please enter 2 keywords:
liberty
slavery

Relevant documents:
1837-VanBuren.txt
1857-Buchanan.txt
1861-Lincoln.txt
1881-Garfield.txt
1889-Harrison.txt

Please enter 2 keywords:
freedom
military

Relevant documents:
1801-Jefferson.txt
1809-Madison.txt
1825-Adams.txt
1829-Jackson.txt
1845-Polk.txt


In [ ]:
# by binary independance model more documents may have same score, its ignoring TF component
# extension to binary independance model

###### Two poisson model

###### Assumption: All documents having almost similiar lengths
###### Relevance score of a document for a query

\begin{equation}
\ rel(D/Q) = \sum_{t \in Q} \dfrac{DF_t(1+k)}{DF_t+k}\times W_t, where 1 \leq k <2
\end{equation}


In [35]:
def get_docs_by_poisson(corpus,num_query_keywords):
    
    # input all the keywords in query
    # preprocess the keywords
    print(f'\nPlease enter {num_query_keywords} keywords:')
    keywords = []
    for i in range(num_query_keywords):
        word = input()
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        word = lemmatizer.lemmatize(word, pos='a')
        keywords.append(word)
        
        
    texts = corpus.values()
    file_ids = list(corpus.keys())
    X1 = vectorizer1.fit_transform(texts)            # binary vectorise
    X2 = vectorizer2.fit_transform(texts)            # count vectorise
    X1 = X1.toarray()
    X2 = X2.toarray()
    features = vectorizer.get_feature_names()
    N = len(corpus)
    k = 1
    
    term_rsv = calculate_rsv_weight(X1, features, N)   # get rsv score 
    id_tfrsv = {} 
    
    for i in range(len(file_ids)):
        tf_rsv_total = 0
        for word in keywords:
            if word in features:
                index = features.index(word)
                if X1[i,index] > 0:                           # check word is in the document
                    rsv = term_rsv[word]
                    tf = X2[i,index]                         # get term frequency
                    tf_rsv = (tf*(k+1)/(tf+k))*rsv           # calculate relavence of a term
                    tf_rsv_total = tf_rsv_total+tf_rsv       # sum all rrelevance score corresponding to all keyword in query
        id_tfrsv[file_ids[i]] = tf_rsv_total
    
    # sort based on relevance score and get best 5 documents
    sorted_tfrsv = sorted(id_tfrsv.items(), key=lambda kv: kv[1], reverse=True)
    best_5_docs = sorted_tfrsv[:5] 
    
    # print best 5 ranked documents
    print('\nRelevant documents:')
    for doc in best_5_docs:
        print(doc[0]) 

In [36]:
get_docs_by_poisson(corpus,num_query_keywords=2)
get_docs_by_poisson(corpus,num_query_keywords=2)
get_docs_by_poisson(corpus,num_query_keywords=2)
get_docs_by_poisson(corpus,num_query_keywords=2)


Please enter 2 keywords:
freedom
jobs

Relevant documents:
2013-Obama.txt
1981-Reagan.txt
2005-Bush.txt
1949-Truman.txt
1985-Reagan.txt

Please enter 2 keywords:
slavery
war

Relevant documents:
1865-Lincoln.txt
1857-Buchanan.txt
1881-Garfield.txt
1821-Monroe.txt
1813-Madison.txt

Please enter 2 keywords:
liberty
slavery

Relevant documents:
2005-Bush.txt
1841-Harrison.txt
1881-Garfield.txt
1857-Buchanan.txt
1901-McKinley.txt

Please enter 2 keywords:
freedom
military

Relevant documents:
1957-Eisenhower.txt
1953-Eisenhower.txt
1825-Adams.txt
1921-Harding.txt
1949-Truman.txt


#### 3. Custom Model

\begin{equation}
\ tf_{t,d} = log(1+f_{t,d})
\\idf_t = \dfrac{N}{1+DF_t}+1
\\ rel(D/Q) = \sum_{t \in Q} tf_{t,d} \times idf_t
\end{equation}

In [37]:
# calculating idf of each term
# arguments are binary vectors of each documents, feature vector and Number of documents as N

def get_ifd(X,features, N):
    term_idf = {}
    for i in range(len(features)):
        DF = sum(X[:,i])                                 #DF of a term sum of column corresponding to the term
        idf = log(N/(1+DF)) + 1
        term_idf[features[i]] = idf
    
    return term_idf

In [42]:
def get_docs_by_custom_model(corpus,num_query_keywords):
    
    # input all the keywords in query
    # preprocess the keywords
    print(f'\nPlease enter {num_query_keywords} keywords:')
    keywords = []
    for i in range(num_query_keywords):
        word = input()
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        word = lemmatizer.lemmatize(word, pos='a')
        keywords.append(word)
        
        
    texts = corpus.values()
    file_ids = list(corpus.keys())
    X1 = vectorizer1.fit_transform(texts)            # binary vectorise
    X2 = vectorizer2.fit_transform(texts)            # count vectorise
    X1 = X1.toarray()
    X2 = X2.toarray()
    features = vectorizer.get_feature_names()
    N = len(corpus)
    
    term_idf = calculate_rsv_weight(X1, features, N)   # get rsv score 
    docid_tfidf = {} 
    
    for i in range(len(file_ids)):
        tfidf_total = 0
        for word in keywords:
            if word in features:
                index = features.index(word)
                if X1[i,index] > 0:                           # check word is in the document
                    idf = term_idf[word]
                    f = X2[i,index]                          # get frequency of term
                    tf = log(1+f)
                    tfidf = tf*idf                         # calculate tfidf
                    tfidf_total = tfidf_total+tfidf       # sum all tfidf corresponding to all keyword in query
        docid_tfidf[file_ids[i]] = tfidf_total
    
    # sort based on relevance score and get best 5 documents
    sorted_tfidf = sorted(docid_tfidf.items(), key=lambda kv: kv[1], reverse=True)
    best_5_docs = sorted_tfidf[:5] 
    
    # print best 5 ranked documents
    print('\nRelevant documents:')
    for doc in best_5_docs:
        print(doc[0]) 

In [44]:
get_docs_by_custom_model(corpus,num_query_keywords=2)
get_docs_by_custom_model(corpus,num_query_keywords=2)
get_docs_by_custom_model(corpus,num_query_keywords=2)
get_docs_by_custom_model(corpus,num_query_keywords=2)


Please enter 2 keywords:
freedom
jobs

Relevant documents:
2005-Bush.txt
1949-Truman.txt
1985-Reagan.txt
1957-Eisenhower.txt
1953-Eisenhower.txt

Please enter 2 keywords:
slavery
war

Relevant documents:
1821-Monroe.txt
1813-Madison.txt
1865-Lincoln.txt
1817-Monroe.txt
1921-Harding.txt

Please enter 2 keywords:
liberty
slavery

Relevant documents:
1841-Harrison.txt
2005-Bush.txt
1881-Garfield.txt
1901-McKinley.txt
1949-Truman.txt

Please enter 2 keywords:
freedom
military

Relevant documents:
2005-Bush.txt
1957-Eisenhower.txt
1949-Truman.txt
1985-Reagan.txt
1953-Eisenhower.txt


In [45]:
get_docs_by_custom_model(corpus,num_query_keywords=4)


Please enter 4 keywords:
military
freedom
missile
war

Relevant documents:
1921-Harding.txt
1825-Adams.txt
1953-Eisenhower.txt
1901-McKinley.txt
1949-Truman.txt


In [46]:
get_docs_by_custom_model(corpus,num_query_keywords=4)


Please enter 4 keywords:
military
war
weapons
missile

Relevant documents:
1813-Madison.txt
1921-Harding.txt
1821-Monroe.txt
1817-Monroe.txt
1865-Lincoln.txt


###### Comparing All models for a query

In [49]:
print('TF-IDF model')
get_relevant_docs_by_tfidf(corpus,num_query_keywords=3) 
print('\nBinary Independance Model')
get_docs_by_bim(corpus,num_query_keywords=3)
print('\nCustom Model')
get_docs_by_custom_model(corpus,num_query_keywords=3)

TF-IDF model

Please enter 3 keywords:
freedom
slavery
war

Relevant documents:
2005-Bush.txt
1865-Lincoln.txt
1813-Madison.txt
1881-Garfield.txt
1953-Eisenhower.txt

Binary Independance Model

Please enter 3 keywords:
freedom
slavery
war

Relevant documents:
1881-Garfield.txt
1889-Harrison.txt
1909-Taft.txt
1953-Eisenhower.txt
1997-Clinton.txt

Custom Model

Please enter 3 keywords:
freedom
slavery
war

Relevant documents:
1921-Harding.txt
1953-Eisenhower.txt
1881-Garfield.txt
1825-Adams.txt
1949-Truman.txt


In [ ]:
# for give set of keywords in the 5 retrieved documents two documents are common

In [50]:
print('TF-IDF model')
get_relevant_docs_by_tfidf(corpus,num_query_keywords=2) 
print('\nBinary Independance Model')
get_docs_by_bim(corpus,num_query_keywords=2)
print('\nCustom Model')
get_docs_by_custom_model(corpus,num_query_keywords=2)

TF-IDF model

Please enter 2 keywords:
freedom
jobs

Relevant documents:
2005-Bush.txt
2013-Obama.txt
1957-Eisenhower.txt
2017-Trump.txt
2009-Obama.txt

Binary Independance Model

Please enter 2 keywords:
freedom
jobs

Relevant documents:
1981-Reagan.txt
1989-Bush.txt
1993-Clinton.txt
1997-Clinton.txt
2009-Obama.txt

Custom Model

Please enter 2 keywords:
freedom
jobs

Relevant documents:
2005-Bush.txt
1949-Truman.txt
1985-Reagan.txt
1957-Eisenhower.txt
1953-Eisenhower.txt


In [ ]:
# for give set of keywords in the 5 retrieved documents two documents are common

In [51]:
print('TF-IDF model')
get_relevant_docs_by_tfidf(corpus,num_query_keywords=4) 
print('\nBinary Independance Model')
get_docs_by_bim(corpus,num_query_keywords=4)
print('\nCustom Model')
get_docs_by_custom_model(corpus,num_query_keywords=4)

TF-IDF model

Please enter 4 keywords:
weapons
missile
war
military

Relevant documents:
1813-Madison.txt
1865-Lincoln.txt
1985-Reagan.txt
1921-Harding.txt
1825-Adams.txt

Binary Independance Model

Please enter 4 keywords:
weapons
military
war
missile

Relevant documents:
1985-Reagan.txt
1949-Truman.txt
1801-Jefferson.txt
1809-Madison.txt
1813-Madison.txt

Custom Model

Please enter 4 keywords:
weapons
military
war
missile

Relevant documents:
1813-Madison.txt
1921-Harding.txt
1821-Monroe.txt
1817-Monroe.txt
1865-Lincoln.txt


In [ ]:
# for give set of keywords both custom model(log nomalised TF-IDf) and TF-IDF retrieved almost similiar documents

In [52]:
print('TF-IDF model')
get_relevant_docs_by_tfidf(corpus,num_query_keywords=2) 
print('\nBinary Independance Model')
get_docs_by_bim(corpus,num_query_keywords=2)
print('\nCustom Model')
get_docs_by_custom_model(corpus,num_query_keywords=2)

TF-IDF model

Please enter 2 keywords:
slavery
war

Relevant documents:
1865-Lincoln.txt
1813-Madison.txt
1857-Buchanan.txt
1821-Monroe.txt
1881-Garfield.txt

Binary Independance Model

Please enter 2 keywords:
slavery
war

Relevant documents:
1837-VanBuren.txt
1857-Buchanan.txt
1861-Lincoln.txt
1865-Lincoln.txt
1881-Garfield.txt

Custom Model

Please enter 2 keywords:
slavery
war

Relevant documents:
1821-Monroe.txt
1813-Madison.txt
1865-Lincoln.txt
1817-Monroe.txt
1921-Harding.txt


In [ ]:
# three documents retrieved almost similiar results

In [53]:
print('TF-IDF model')
get_relevant_docs_by_tfidf(corpus,num_query_keywords=2) 
print('\nBinary Independance Model')
get_docs_by_bim(corpus,num_query_keywords=2)
print('\nCustom Model')
get_docs_by_custom_model(corpus,num_query_keywords=2)

TF-IDF model

Please enter 2 keywords:
freedom
military

Relevant documents:
2005-Bush.txt
1957-Eisenhower.txt
1949-Truman.txt
1985-Reagan.txt
1953-Eisenhower.txt

Binary Independance Model

Please enter 2 keywords:
freedom
military

Relevant documents:
1801-Jefferson.txt
1809-Madison.txt
1825-Adams.txt
1829-Jackson.txt
1845-Polk.txt

Custom Model

Please enter 2 keywords:
freedom 
military

Relevant documents:
1825-Adams.txt
1857-Buchanan.txt
1921-Harding.txt
1829-Jackson.txt
1833-Jackson.txt


In [ ]:
# No common documents found in all three models